In [1]:
### Import packages ###
import time
import numpy as np
import math as math
import pandas as pd
import random as random

### Append Path ###
import sys
sys.path.append('..')

### Import functions ###
from utils.Main import *
from utils.Selector import *
from utils.Auxiliary import *
from utils.Prediction import *

# Inputs

In [2]:
### Get Directory ###
cwd = os.getcwd()
ParentDirectory = os.path.abspath(os.path.join(cwd, "..",".."))

### DataType ###
DataFileInput = "BankNote"  # BankNote  Bar7  BreastCancer  CarEvaluation  COMPAS  FICO  Haberman  Iris  MONK1  MONK3

In [3]:
SimulationConfigInput = {'DataFileInput': DataFileInput,
    'Seed': 1,
    'TestProportion': 0.2,
    'CandidateProportion': 0.8,
    'SelectorType': 'BatchQBCSelector', # PassiveLearningSelector, BALDSelector, BatchQBCSelector
    'ModelType': 'LFRPredictor', # RandomForestClassifierPredictor, BayesianNeuralNetworkPredictor, GaussianProcessClassifierPredictor, TreeFarmsPredictor, LFRPredictor
    'UniqueErrorsInput': 0,
    'n_estimators': 100,
    'regularization': 0.01,
    'RashomonThresholdType': "Adder",
    'RashomonThreshold': 0.1,
    'Type': 'Classification',
    "DiversityWeight" : 0,
    "DensityWeight" : 0,
    "BatchSize" : 7,
    "auto_tune_epsilon" : False}
Results = OneIterationFunction(SimulationConfigInput)


Iteration: 0
null
treefarms reported successful execution
training completed. Number of trees in the Rashomon set: 25
Finding Optimal Objective...
{
  "false": {
    "false": {
      "complexity": 0.009999999776482582,
      "loss": 0.004566209856420755,
      "name": "Y",
      "prediction": 0
    },
    "feature": 0,
    "name": "variance_leq_-3",
    "reference": 1,
    "relation": "==",
    "true": {
      "complexity": 0.009999999776482582,
      "loss": 0.0,
      "name": "Y",
      "prediction": 1
    },
    "type": "integral"
  },
  "feature": 5,
  "model_objective": 0.09022830426692963,
  "name": "skewness_leq_5",
  "reference": 1,
  "relation": "==",
  "true": {
    "false": {
      "complexity": 0.009999999776482582,
      "loss": 0.013698630034923553,
      "name": "Y",
      "prediction": 0
    },
    "feature": 3,
    "name": "variance_leq_1",
    "reference": 1,
    "relation": "==",
    "true": {
      "complexity": 0.009999999776482582,
      "loss": 0.0319634675979614

In [7]:
Results["RefitDecisionVec"]

,RefitDecision
0,1
1,1
2,1
3,1
4,1
...,...
121,1
122,1
123,1
124,1


In [5]:
Results["EpsilonVec"]

,Epsilon
0,0.1
1,0.1
2,0.1
3,0.1
4,0.1
...,...
121,0.1
122,0.1
123,0.1
124,0.1


In [6]:
Results["ErrorVec"]

,Error
0,0.898182
1,0.909091
2,0.909091
3,0.912727
4,0.912727
...,...
121,0.923636
122,0.923636
123,0.923636
124,0.923636
